### **Project One: Small Language Model (Predictive Text)**
Use an http API to populate data to a complex data structure
- develop a small computational experiment
- write a well-organized, self-contained, easy-to-read computational notebook
- present a visualization of the experimental results in graphical form

21-26 February, David Lu

In [2]:
import re
import random
import requests
import networkx as nx
import collections

#### **Part One: Input**

The input text for this model is the preface to Livy's *The Early History of Rome (Books 1-5)*, as translated by Aubrey de Sélincourt. The text has been uploaded to the project's Github directory for easy access, and simple Markdown formatting has been applied for improved readability.

Getting started, the  full Markdown text is retrieved with help from the `requests` library.
Next, the text is separated into its constituent words, which are the basic unit of manipulation of the model.


In [3]:
address = 'https://raw.githubusercontent.com/zhdavidlu/cap-comp215/main/projects/livy.md'

response = requests.get(address)
text = response.text  # full markdown text

remove = re.sub(r'.*\n\n', '', text, count = 1)  # remove the heading
replace = re.sub(r'\n', ' ', remove)             # replace line breaks with spaces

words = replace.split()  # split the text into a list of words
words[:9]  # show the first nine words, for a sense of what the list looks like

['The', 'task', 'of', 'writing', 'a', 'history', 'of', 'our', 'nation']

#### **Part Two: Restructuring**

The prediction mechanism of the model is extremely simple: provided a pair of consecutive words from the input, the prediction is taken to be the next word to appear after the pair. Importantly, if the pair appears more than once in the input, then the prediction will be a random selection from the various words to follow the pair throughout the text.

To make this selection in a timely manner, the word or words that follow any given pair needs to be retrieved efficiently. This is achieved, as was suggested, by a combination of the `defaultdict` sub-class and the "sliding-window algorithm". Each unique pair of consecutive words is placed within the dictionary `dictionnaire` as a key, and the word or list of words which follow are their corresponding values.

In [4]:
dictionnaire = collections.defaultdict(list)
triplets = [((words[i], words[i+1]), words[i+2]) for i in range(len(words)-2)]  # "sliding window algorithm"

for k,v in triplets:
    dictionnaire[k].append(v)

list(dictionnaire.items())[:3]  # the first three entries in the dictionary

[(('The', 'task'), ['of']),
 (('task', 'of'), ['writing']),
 (('of', 'writing'), ['a'])]

#### **Part Three: Making Predictions**

A relatively straightforward implementation of the aforementioned prediction mechanism.

In [ ]:
# Helper functions

# Picks the first two words of a random sentence from the
# text to be the starting point for the model

def random_start (words):

    while True:
        random_number = random.randrange(0, len(words)-1)
        if words[random_number][-1] == '.': break

    return (words[random_number+1], words[random_number+2])


# Breaks up very long lines of text into multiple shorter ones
# to improve readability (default width ~ 65 chars)

def break_line (line, linewidth = 65):

    if len(line) < (5/4)*linewidth:
        return line

    else:
        formatted_line = []
        former = 0
        additional_characters = 0

        for index in range(linewidth, len(line), linewidth):
            index += additional_characters

            while index < len(line) and line[index] != ' ':
                index += 1
                additional_characters += 1

            formatted_line.append(line[former:index] + '\n')
            former = index + 1

        formatted_line.append(line[former:])

    return ''.join(formatted_line)

# test_lnstr = ''.join(['0 '] * int(65 * (5/8)+1))
# print(break_line(test_lnstr))

In [ ]:
# Top-level / User-oriented function 1
def write_words (words, table, num_words = 12):

    words_in = random_start(words)
    word_sequence = list(words_in)

    for i in range(num_words):
        word_out = random.choice(dictionnaire[words_in])
        words_in = (words_in[1], word_out)
        word_sequence.append(word_out)

    sentence = ' '.join(word_sequence)
    if sentence[-1] not in [';', '.']:  # add some nice ellipsis if the word
        sentence = f'{sentence}...'     # sequence ends midway through a sentence

    return sentence

# Top-level / User-oriented function 2
def write_sentences (words, table, num_sentences = 3):

    sentence_count = 0
    words_in = random_start(words)
    word_sequence = list(words_in)

    while sentence_count < num_sentences:
        word_out = random.choice(dictionnaire[words_in])
        words_in = (words_in[1], word_out)
        word_sequence.append(word_out)

        if word_out[-1] == '.':
            sentence_count += 1

    return break_line(' '.join(word_sequence))

# Merge into one function

In [ ]:
some_words = write_words(words, dictionnaire)
some_sentences = write_sentences(words, dictionnaire)

print(some_words)
print()
print(some_sentences)

Turnus and his people, in their anxiety for the future, then looked for help...

Hitherto, however, he had been exposed was left high and dry by the
insult of having to step down in favour of a greater future. He went
first to Macedonia, then in his escape from the vices of avarice
and luxury; nowhere have thrift and plain living been for so long
held in such esteem. Indeed, poverty, with us, went hand in hand
with contentment.


In [ ]:
# helpful functions and their interfaces:
# ## re.search(<pattern>, <string>)
# ## random.choices(<list>)


# more ideas
# ## a weighted networkx graph showing the connections between the most common words (eg. pairs with more than three successors)
# ## user-oriented function that lets the user choose between sequences with a certain number of words or sentences
# #### for sentences, also choose whether sentences should be short, long, or intermediate

#### **Visualisations**

In [ ]:
# Visualisation function from a past project,
# imported here for style references, needs to be adapted

# def draw_shortest_paths(G, T, node_positions, edge_weights):

#     # T_complement = nx.difference(G, T)

#     node_labels = {node: f'${node}$' for node in G.nodes()}

#     node_style = {'node_size': 600, 'node_color': 'tab:blue', 'alpha': 0.9}
#     label_style = {'labels': node_labels, 'font_color': 'white'}  # font_size

#     nx.draw_networkx_nodes(G, node_positions, **node_style)
#     nx.draw_networkx_labels(G, node_positions, **label_style)
#     nx.draw_networkx_edge_labels(G, node_positions, edge_weights)

#     return 0

# Try a tree-like representation of a short word sequence
# Possibility at each "stage" depends on pair (previous two states)